Importar la librerias y el archivo

In [2]:
import pandas as pd
import numpy as np

###leer archivo de excel
datos = pd.read_csv("C:/Users/Aleja/Documents/Maestria/DMEyF/DMEyF2024/Competencia_03/competencia_03_crudo.csv.gz")
datosO = datos.copy()

Agregar nueva columna de fecha

In [4]:
# Convertir la columna periodo a fecha
datos['fecha'] = pd.to_datetime(datos['foto_mes'], format='%Y%m')

# calcular los meses siguientes
datos['mes_1'] = (datos['fecha'] + pd.DateOffset(months=1)).dt.strftime('%Y%m').astype(int)
datos['mes_2'] = (datos['fecha'] + pd.DateOffset(months=2)).dt.strftime('%Y%m').astype(int)

Hacer los cruces respectivos

In [5]:
###dejar solo dos columnas del dataset2
datosO = datosO[['numero_de_cliente','foto_mes']]

###hacer los cruces
combinacion = pd.merge(datos, datosO, left_on=['numero_de_cliente','mes_1'], right_on=['numero_de_cliente','foto_mes'], how = 'left')
combinacion = pd.merge(combinacion, datosO, left_on=['numero_de_cliente','mes_2'], right_on=['numero_de_cliente','foto_mes'], how = 'left')


Crear clase ternaria

In [7]:
condiciones = [
    (combinacion['foto_mes_y']>0) & (combinacion['foto_mes']>0),
    (combinacion['foto_mes_y']>0) & (combinacion['foto_mes'].isnull()) & (combinacion['foto_mes_x'] != 202108),
    (combinacion['foto_mes_y'].isnull()) & (combinacion['foto_mes'].isnull()) & (combinacion['foto_mes_x'] != 202109),
    (combinacion['foto_mes_y'].isnull()) & (combinacion['foto_mes']>0)
]

respuesta = ['CONTINUA', 'BAJA+2', 'BAJA+1', 'CONTINUA']

combinacion['clase_ternaria'] = np.select(condiciones, respuesta, default = '')

Eliminar columnas, renombrar columna

In [8]:
###eliminar columnas
combinacion = combinacion.drop(columns=['foto_mes', 'foto_mes_y','mes_1','mes_2','fecha'])

###actualizar el nombre de una columna
combinacion = combinacion.rename(columns={'foto_mes_x': 'foto_mes'})


Revisar

In [14]:
c = combinacion.groupby(['foto_mes','clase_ternaria']).size()
c

foto_mes  clase_ternaria
201901    BAJA+2               718
          CONTINUA          123346
201902    BAJA+2               688
          CONTINUA          124391
201903    BAJA+2               760
                             ...  
202106    CONTINUA          162894
202107    BAJA+2              1294
          CONTINUA          162783
202108                      164146
202109                      165644
Length: 64, dtype: int64

Quitar los baja + 1

In [12]:
combinacion = combinacion[combinacion['clase_ternaria'] != 'BAJA+1']

In [15]:
combinacion.to_csv("C:/Users/Aleja/Documents/Maestria/DMEyF/DMEyF2024/Competencia_03/dataset_SinBAJA1C3.csv.gz", index = False)